## Q1. Knowing Docker Tags

Which tag has the following text? - Automatically remove the container when it exits

- `--delete`
- `--rc`
- `--rmc`
- **`--rm`**


## Q2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use ```pip list``` ). 

What is version of the package *wheel* ?

- 0.42.0
- 1.0.0
- 23.0.1
- 58.1.0

In [ ]:
!docker run --rm -it --entrypoint bash python:3.9 -c "pip show wheel | grep Version"

Version: 0.42.0

# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from September 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz```

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)

## Q3. Count records 

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 15767
- 15612
- 15859
- 89009

```sql
SELECT COUNT(*) AS total_trips
FROM taxi_trips
WHERE DATE(lpep_pickup_datetime) = '2019-09-18' AND DATE(lpep_dropoff_datetime) = '2019-09-1';


**15612**

## Q4. Largest trip for each day

Which was the pick up day with the largest trip distance
Use the pick up time for your calculations.

- 2019-09-18
- 2019-09-16
- 2019-09-26
- 2019-09-21

```sql
SELECT 
  CAST(lpep_pickup_datetime AS DATE) AS pick_up_day, 
  MAX(trip_distance) AS largest_trip_distance
FROM green_taxi_data
GROUP BY pick_up_day
ORDER BY largest_trip_distance DESC
LIMIT 10;

```sql
| pick_up_day | largest_trip_distance |
|--------------|------------------------|
| 2019-09-26   | 341.64                 |
| 2019-09-21   | 135.53                 |
| 2019-09-16   | 114.3                  |
| 2019-09-28   | 89.64                  |
| 2019-09-24   | 82.12                  |
| 2019-09-18   | 70.28                  |
| 2019-09-10   | 69.67                  |
| 2019-09-27   | 68.41                  |
| 2019-09-02   | 61.69                  |
| 2019-09-19   | 61.26                  |


## Q5. Three biggest pick up Boroughs

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
 
- "Brooklyn" "Manhattan" "Queens"
- "Bronx" "Brooklyn" "Manhattan"
- "Bronx" "Manhattan" "Queens" 
- "Brooklyn" "Queens" "Staten Island"

```sql
SELECT z."Borough", SUM(t.total_amount) AS total_amount_sum
FROM green_taxi_data t
JOIN zones z ON t."PULocationID" = z."LocationID"
WHERE DATE(t.lpep_pickup_datetime) = '2019-09-18' AND z."Borough" != 'Unknown'
GROUP BY z."Borough"
HAVING SUM(t.total_amount) > 50000
ORDER BY total_amount_sum DESC
LIMIT 3;

```sql
"Borough"	"total_amount_sum"
"Brooklyn"	96333.24000000124
"Manhattan"	92271.30000000076
"Queens"	78671.71000000054

## Q6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- JFK Airport
- Long Island City/Queens Plaza

```sql
SELECT
  DATE(t.lpep_pickup_datetime) AS pickup_day,
  z_pickup."Zone" AS pickup_zone,
  z_dropoff."Zone" AS dropoff_zone,
  t.tip_amount AS tip_amount
FROM
  green_taxi_data t
LEFT JOIN
  zones z_pickup ON t."PULocationID" = z_pickup."LocationID"
LEFT JOIN
  zones z_dropoff ON t."DOLocationID" = z_dropoff."LocationID"
WHERE
  date_trunc('month', t.lpep_pickup_datetime) = '2019-09-01'
  AND z_pickup."Zone" = 'Astoria'
ORDER BY
  t.tip_amount DESC
LIMIT 1;

```sql
"pickup_day"	"pickup_zone"	"dropoff_zone"	"tip_amount"
"2019-09-08"	"Astoria"	"JFK Airport"	62.31

## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. 
Copy the files from the course repo
[here](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/01-docker-terraform/1_terraform_gcp/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.


## Question 7. Creating Resources

After updating the main.tf and variable.tf files run:

```
terraform apply
```